In [18]:
import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from statsmodels.formula.api import ols
from scipy.stats import gaussian_kde
import scipy
import scipy.sparse
import patsy
from statistics import median
import bz2
import math
import os
from pathlib import Path

ModuleNotFoundError: No module named 'pandas'

In [ ]:
model_dir= './'
def sort_cols(test):
    return(test.reindex(sorted(test.columns), axis=1))
frames = {}
for year in [2004,2005,2006]:
    fil = "./data/" + "pandas-frames." + str(year) + ".pickle.bz2"
frames.update(pd.read_pickle(fil))
for x in frames:
    frames[x]= sort_cols(frames[x])
covariance = {}
for year in [2003,2004,2005,2006]:
    fil ="./data/" + "covariance." + str(year) + ".pickle.bz2"
covariance.update(pd.read_pickle(fil))
industry_factors= ['AERODEF', 'AIRLINES', 'ALUMSTEL', 'APPAREL', 'AUTO',
'BANKS','BEVTOB', 'BIOLIFE', 'BLDGPROD','CHEM', 'CNSTENG',
'CNSTMACH', 'CNSTMATL', 'COMMEQP', 'COMPELEC',
'COMSVCS', 'CONGLOM', 'CONTAINR', 'DISTRIB',
'DIVFIN', 'ELECEQP', 'ELECUTIL', 'FOODPROD', 'FOODRET', 'GASUTIL',
'HLTHEQP', 'HLTHSVCS', 'HOMEBLDG', 'HOUSEDUR','INDMACH', 'INSURNCE',
'INTERNET',
'LEISPROD', 'LEISSVCS', 'LIFEINS', 'MEDIA', 'MGDHLTH','MULTUTIL',
'OILGSCON', 'OILGSDRL', 'OILGSEQP', 'OILGSEXP',
'PAPER', 'PHARMA', 'PRECMTLS','PSNLPROD','REALEST',
'RESTAUR', 'ROADRAIL','SEMICOND', 'SEMIEQP','SOFTWARE',
'SPLTYRET', 'SPTYCHEM', 'SPTYSTOR', 'TELECOM', 'TRADECO', 'TRANSPRT',
'WIRELESS']
style_factors= ['BETA','SIZE','MOMENTUM','VALUE','LEVERAGE','LIQUIDTY']
def wins(x,a,b):
    return(np.where(x <= a,a, np.where(x >= b, b, x)))
def clean_nas(df):
    numeric_columns= df.select_dtypes(include=[np.number]).columns.tolist()
    for numeric_column in numeric_columns:
        df[numeric_column]= np.nan_to_num(df[numeric_column])
        return df
def density_plot(data, title):
    density= gaussian_kde(data)
    xs = np.linspace(np.min(data),np.max(data),200)
    density.covariance_factor= lambda : .25
    density._compute_covariance()
    plt.plot(xs,density(xs))
    plt.title(title)
    plt.show()
    test = frames['20040102']
    density_plot(test['Ret'], 'Daily return pre-winsorization')
    density_plot(wins(test['Ret'],-0.2,0.2), 'Daily return winsorized')
    D = (test['SpecRisk'] / (100 * math.sqrt(252))) ** 2
    density_plot(np.sqrt(D), 'SpecRisk')
def get_estu(df):
    """Estimation universe definition"""
    estu = df.loc[df.IssuerMarketCap > 1e9].copy(deep=True)
    return estu
def colnames(X):
    """ return names of columns, for DataFrame or DesignMatrix """
    if(type(X)== patsy.design_info.DesignMatrix):
        return(X.design_info.column_names)
    if(type(X)== pandas.core.frame.DataFrame):
        return(X.columns.tolist())
    return(None)
def diagonal_factor_cov(date, X):
    """Factor covariance matrix, ignoring off-diagonal for simplicity"""
    cv = covariance[date]
    k = np.shape(X)[1]
    Fm = np.zeros([k,k])
    for j in range(0,k):
        fac = colnames(X)[j]
    Fm[j,j]= (0.01**2) * cv.loc[(cv.Factor1==fac) & (cv.Factor2==fac),"VarCovar"]
    return(Fm)
def risk_exposures(estu):
    """Exposure matrix for risk factors, usually called X in class"""
    L = ["0"]
    L.extend(style_factors)
    L.extend(industry_factors)
    my_formula= " + ".join(L)
    return patsy.dmatrix(my_formula, data = estu)

In [ ]:
my_date= '20040102'
# estu = estimation universe
estu = get_estu(frames[my_date])
estu['Ret']= wins(estu['Ret'],-0.25, 0.25)
rske = risk_exposures(estu)
F = diagonal_factor_cov(my_date, rske)
X = np.asarray(rske)
D = np.asarray( (estu['SpecRisk'] / (100 * math.sqrt(252))) ** 2 )
kappa= 1e-5
candidate_alphas= [
'STREVRSL', 'LTREVRSL', 'INDMOM',
'EARNQLTY', 'EARNYILD', 'MGMTQLTY', 'PROFIT', 'SEASON', 'SENTMT']

Problem 1

In [ ]:
for date in frames:
    estu = get_estu(frames[date])
    estu['Ret']= wins(estu['Ret'],-0.25, 0.25)
    X = np.asarray(risk_exposures(estu))
    Ret = np.asarray(estu['Ret'])
    X_plus= np.linalg.pinv(X)
    risk_model_explanation= X @ (X_plus @ Ret)
    Y = Ret- risk_model_explanation
    estu['Y']= Y
    frames[date]= estu

Problem 2

In [ ]:
#Problem 2
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
sorted_dates= sorted(list(frames.keys()))
split_point= int(len(sorted_dates) * 0.7)
train_dates= sorted_dates[:split_point]
test_dates= sorted_dates[split_point:]
print(f"Training on {len(train_dates)} dates. Testing on {len(test_dates)} dates.")
train_panel= pd.concat([frames[d] for d in train_dates])
test_panel= pd.concat([frames[d] for d in test_dates]) # The Vault
train_clean= train_panel.dropna(subset=candidate_alphas + ['Y'])
X_train= train_clean[candidate_alphas]
y_train= train_clean['Y']
tscv = TimeSeriesSplit(n_splits=3)
print("\nTraining Linear Model (Lasso)...")
lasso = LassoCV(cv=tscv, random_state=42).fit(X_train, y_train)
lasso_score= lasso.score(X_train, y_train) # R^2 score
print(f"Lasso Best Alpha: {lasso.alpha_:.6f}")
print("\nTraining Random Forest...")
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_params= {
'n_estimators': [50, 100],
'max_depth': [3, 5],
'min_samples_leaf': [100]
}
rf_grid= GridSearchCV(rf, rf_params, cv=tscv, scoring='neg_mean_squared_error')
rf_grid.fit(X_train, y_train)
print(f"Best RF Params: {rf_grid.best_params_}")
print("\nTraining Gradient Boosting...")
gb= GradientBoostingRegressor(random_state=42)
gb_params= {
'n_estimators': [50, 100],
'learning_rate': [0.01, 0.1], # Key parameter for Boosting
'max_depth': [3], # Boosting prefers shallow trees
'min_samples_leaf': [100]
}
gb_grid= GridSearchCV(gb, gb_params, cv=tscv, scoring='neg_mean_squared_error')
gb_grid.fit(X_train, y_train)
print(f"Best GB Params: {gb_grid.best_params_}")
print("\n--- Model Comparison (MSE - Lower is Better) ---")
lasso_mse= mean_squared_error(y_train, lasso.predict(X_train)) # Simple MSE for Lasso
rf_mse= -rf_grid.best_score_
gb_mse= -gb_grid.best_score_
print(f"Lasso MSE: {lasso_mse:.6f}")
print(f"Random Forest CV MSE: {rf_mse:.6f}")
print(f"Gradient Boosting CV MSE: {gb_mse:.6f}")
best_score= min(lasso_mse, rf_mse, gb_mse)
if best_score== lasso_mse:
    print("\nWINNER: Lasso (Linear)")
    final_model= lasso
elif best_score== rf_mse:
    print("\nWINNER: Random Forest")
    final_model= rf_grid.best_estimator_
else:
    print("\nWINNER: Gradient Boosting")
    final_model= gb_grid.best_estimator_

Problem 3

In [ ]:
def woodbury_inverse(X, F, D, eps=1e-8):
    n, p = X.shape
    D = np.maximum(np.nan_to_num(D, nan=eps, posinf=eps, neginf=eps), eps)
    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    F = np.nan_to_num(F, nan=0.0, posinf=0.0, neginf=0.0)
    D_inv= 1.0 / D
    F_reg= F + eps * np.eye(p)
    try:
        F_inv= np.linalg.inv(F_reg)
    except np.linalg.LinAlgError:
        F_inv= np.linalg.pinv(F_reg)
        XtDinv = X.T * D_inv
        XtDinvX = XtDinv @ X
        middle = F_inv + XtDinvX
    try:
        middle_inv= np.linalg.inv(middle)
    except np.linalg.LinAlgError:
        middle_inv= np.linalg.pinv(middle)
        DinvX = D_inv[:, None] * X
    def apply(v):
        v = np.nan_to_num(v, nan=0.0, posinf=0.0, neginf=0.0)
        term1 = D_inv * v
        term2 = DinvX @ (middle_inv @ (X.T @ term1))
        return term1- term2
    return apply

Problem 4

In [ ]:
lambda_risk= 0.01
position_limit= 0.05
min_variance= 1e-6
initial_capital= 1_000_000
backtest_results= []
for date in test_dates:
    if date not in frames:
        continue
    try:
        estu = frames[date]
        if len(estu) < 50:
            continue
        X_alpha= estu[candidate_alphas].fillna(0)
        alpha= final_model.predict(X_alpha)
        alpha= np.nan_to_num(alpha, nan=0.0, posinf=0.0, neginf=0.0)
        if np.sum(np.abs(alpha)) < 1e-10:
            continue
        X_risk= np.asarray(risk_exposures(estu))
        F_risk= diagonal_factor_cov(date, risk_exposures(estu))
        D_risk= np.asarray((estu['SpecRisk'] / (100 * math.sqrt(252))) ** 2)
        sigma_inv= woodbury_inverse(X_risk, F_risk, D_risk, min_variance)
        h = (1.0 / lambda_risk) * sigma_inv(alpha)
        if np.any(np.isnan(h)) or np.any(np.isinf(h)):
            continue
        h = h- np.mean(h)
        h = wins(h,
        -position_limit, position_limit)
        gross = np.sum(np.abs(h))
        if gross <= 1e-10:
            continue
        h /= gross
        ret = np.nan_to_num(estu['Ret'].values, nan=0.0, posinf=0.0, neginf=0.0)
        pnl= np.dot(h, ret)
        hX = h @ X_risk
        factor_var= float(hX @ F_risk @ hX.T)
        idio_var= float(np.dot(h * h, D_risk))
        total_var= factor_var + idio_var
        total_risk= math.sqrt(max(0, total_var))
        factor_risk= math.sqrt(max(0, factor_var))
        idio_risk= math.sqrt(max(0, idio_var))
        backtest_results.append({
            'date': date,
            'pnl': pnl,
            'long_mv': float(np.sum(h[h > 0])),
            'short_mv': float(np.abs(np.sum(h[h < 0]))),
            'daily_risk_dollars': total_risk * initial_capital,
            'daily_factor_risk_dollars': factor_risk * initial_capital,
            'daily_idio_risk_dollars': idio_risk * initial_capital,
            'idio_risk_pct': 100 * idio_risk / total_risk if total_risk > 0 else
            'num_positions': int(np.sum(np.abs(h) > 1e-6))
            })
    except Exception:
        continue
results_df= pd.DataFrame(backtest_results)
results_df['cumulative_pnl']= results_df['pnl'].cumsum()
results_df['trading_day']= np.arange(len(results_df))

In [ ]:
total_pnl= results_df['pnl'].sum()
avg_daily_pnl= results_df['pnl'].mean()
vol_daily_pnl= results_df['pnl'].std()
sharpe= avg_daily_pnl / vol_daily_pnl * np.sqrt(252)
total_pnl, sharpe

Plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, axes = plt.subplots(3, 2, figsize=(16, 12))
axes[0, 0].plot(results_df['trading_day'], results_df['long_mv'])
axes[0, 0].fill_between(results_df['trading_day'], 0, results_df['long_mv'])
axes[0, 0].set_title('Long Market Value')
axes[0, 1].plot(results_df['trading_day'], results_df['short_mv'])
axes[0, 1].fill_between(results_df['trading_day'], 0, results_df['short_mv'])
axes[0, 1].set_title('Short Market Value')
axes[1, 0].plot(results_df['trading_day'], results_df['cumulative_pnl'])
axes[1, 0].fill_between(results_df['trading_day'], 0, results_df['cumulative_pnl'])
axes[1, 0].axhline(0)
axes[1, 0].set_title('Cumulative Profit')
axes[1, 1].plot(results_df['trading_day'], results_df['daily_risk_dollars'])
axes[1, 1].fill_between(results_df['trading_day'], 0, results_df['daily_risk_dollars'])
axes[1, 1].set_title('Daily Portfolio Risk (Dollars)')
axes[2, 0].plot(results_df['trading_day'], results_df['idio_risk_pct'])
axes[2, 0].fill_between(results_df['trading_day'], 0, results_df['idio_risk_pct'])
axes[2, 0].axhline(50)
axes[2, 0].set_ylim(0, 100)
axes[2, 0].set_title('Percent of Risk that is Idiosyncratic')
axes[2, 1].plot(results_df['trading_day'], results_df['daily_factor_risk_dollars'])
axes[2, 1].plot(results_df['trading_day'], results_df['daily_idio_risk_dollars'])
axes[2, 1].set_title('Risk Decomposition (Dollars)')
plt.tight_layout()
plt.show()
